## Getting Started

In [60]:
!pip install requests
!pip install beautifulsoup4
!pip install pyarrow

import requests
from bs4 import BeautifulSoup
import pandas as pd
import ssl
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import re
nltk.download('vader_lexicon')


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/katytseng/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [61]:
url = "https://my.sa.ucsb.edu/public/curriculum/coursesearch.aspx"
data = requests.get(url)
html = data.text
soup = BeautifulSoup(html, 'html.parser')
session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded'
}

In [62]:
classId = []
year = []
level = []

for option in soup.find_all('option'):
    classId.append(option['value'])

year = classId[99:108]
level = classId[-3:]
classId = classId[:99]

depAndQuarter = []

for i in year:
    for c in classId:
        depAndQuarter.append([c, i])

depAndQuarter = pd.DataFrame(depAndQuarter)
depAndQuarter = depAndQuarter.rename(columns={0: "department", 1: "quarter"})
depAndQuarter = depAndQuarter[~depAndQuarter.isin(['20251']).any(axis=1)]
depAndQuarter = depAndQuarter.sort_values(by='department')
departments = depAndQuarter.groupby(['department']).count()

In [20]:
depAndQuarter.iloc[2, 1]

'20234'

In [64]:
#Loop through the dropdown choices
data = pd.DataFrame(columns=['CourseID','Department','CourseTitle', 'Days','Time','Location'])

for d in range(len(departments)):
    for q in range(9):
        response = session.get(url, headers=headers)
        if response.status_code != 200:
            print("Error loading initial page:", response.status_code)
        else:
            # Parse the initial HTML content
            soup = BeautifulSoup(response.content, 'html.parser')
        
            # Extract the necessary hidden fields for the form submission
            viewstate = soup.find("input", {"id": "__VIEWSTATE"})['value']
            viewstategenerator = soup.find("input", {"id": "__VIEWSTATEGENERATOR"})['value']
            eventvalidation = soup.find("input", {"id": "__EVENTVALIDATION"})['value']
        
            # Prepare the form data
            form_data = {
                '__VIEWSTATE': viewstate,
                '__VIEWSTATEGENERATOR': viewstategenerator,
                '__EVENTVALIDATION': eventvalidation,
                'ctl00$pageContent1$courseList': departments.index[d],  # Subject Area: Anthropology
                'ctl00$pageContent1$quarterList': depAndQuarter.iloc[q, 1],  # Quarter: FALL 2024
                'ctl00$pageContent1$dropDownCourseLevels': 'Undergraduate',  # Course Level: Undergraduate
                'ctl00$pageContent1$searchButton.x': '0',  # Simulating button click coordinates
                'ctl00$pageContent1$searchButton.y': '0'
            }
        
            # Send the POST request with form data to simulate clicking the search button
            response = session.post(url, data=form_data, headers=headers)
        
            # Check if the request was successful
            if response.status_code == 200:
                # Parse the response content using BeautifulSoup
                soup = BeautifulSoup(response.content, 'html.parser')
            else:
                print("Error in POST request:", response.status_code)

        classList = soup.find_all("tr", {"class": "CourseInfoRow"})

        for c in range(len(classList)):
            class_vec = []
            course = classList[c]
            class_vec.append(course.find_all("td", {"id": "CourseTitle"})[0].contents[0].strip())
            class_vec.append(departments.index[d])
            class_vec.append(course.find_all("td", {"class": "PrimaryCourse"})[0].find("span").get_text().strip())
            courseInfo = course.find_all("td")[-5:-2]
            for k in courseInfo:
                class_vec.append(k.get_text().strip())
            data.loc[len(data)] = class_vec
    
    print(departments.index[d], ' is department ', d, ' of ', len(departments), '\n')

ANTH  is department  0  of  98 

ART  is department  1  of  98 

ART  CS   is department  2  of  98 

ARTHI  is department  3  of  98 

ARTST  is department  4  of  98 

AS AM  is department  5  of  98 

ASTRO  is department  6  of  98 

BIOE  is department  7  of  98 

BIOL  is department  8  of  98 

BIOL CS   is department  9  of  98 

BL ST  is department  10  of  98 

C LIT  is department  11  of  98 

CH E  is department  12  of  98 

CH ST  is department  13  of  98 

CHEM  is department  14  of  98 

CHEM CS   is department  15  of  98 

CHIN  is department  16  of  98 

CLASS  is department  17  of  98 

CMPSC  is department  18  of  98 

CMPSCCS   is department  19  of  98 

CMPTG  is department  20  of  98 

CMPTGCS   is department  21  of  98 

CNCSP  is department  22  of  98 

COMM  is department  23  of  98 

DANCE  is department  24  of  98 

DYNS  is department  25  of  98 

EACS  is department  26  of  98 

EARTH  is department  27  of  98 

ECE  is department  28  of

In [65]:
data.to_csv("data.csv", index=False) 

In [112]:
#fill in the section course titles
'''
for c in range(len(classList)):
    class_vec = []
    course = classList[c]
    class_vec.append(course.find_all("td", {"id": "CourseTitle"})[0].contents[0].strip())
    class_name = course.find_all("td", {"class": "PrimaryCourse"})[0].find("span").get_text().strip()
    if len(class_name) != 0:
        class_vec.append(class_name)
        className = class_name
    else:
        class_vec.append(className)
    courseInfo = course.find_all("td")[-5:-2]
    for k in courseInfo:
        class_vec.append(k.get_text().strip())


'''


'\nfor c in range(len(classList)):\n    class_vec = []\n    course = classList[c]\n    class_vec.append(course.find_all("td", {"id": "CourseTitle"})[0].contents[0].strip())\n    class_name = course.find_all("td", {"class": "PrimaryCourse"})[0].find("span").get_text().strip()\n    if len(class_name) != 0:\n        class_vec.append(class_name)\n        className = class_name\n    else:\n        class_vec.append(className)\n    courseInfo = course.find_all("td")[-5:-2]\n    for k in courseInfo:\n        class_vec.append(k.get_text().strip())\n\n\n'

In [66]:

dep_fullnames = pd.DataFrame(columns=["DepartmentCode", "DepartmentName"])
nameList = []
for option in soup.find_all('option'):
    nameList.append(option.text)
for d in range(len(nameList)):
    dep_fullnames.loc[d] = [nameList[d].split('-')[-1].strip(), nameList[d].split('-')[0].strip()]

pd.DataFrame(dep_fullnames).to_csv("depNames.csv", index=False) 